In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('Data/eda_receitas_data/receitas.json')

# Sistema de recomendação

Para criar um sistema de recomendação, o cliente irá informar 5 tags representando suas preferências. Essas tags irão ser comparadas com as tags das receitas.

As receitas com maior similaridade com as tags informadas pelo cliente serão recomendadas. A forma de calcular similaridade utilizada foi a similaridade de cosseno. As tags são vetorizadas por meio do objeto `TfidVectorizer` da biblioteca `scikit-learn` (https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction). As palavras são vetorizadas, tem seus pesos calibrados e então a distância entre os vetores das tags do cliente e das receitas é calculada no espaço vetorial, utilizando o método `cosine_similarity`.

In [33]:
# Função para criar o perfil do cliente
def criar_perfil(tags):
    perfil = ' '.join(tags)
    return perfil

def recomendar_receitas(cliente_perfil, df_receitas, top_n=5):
    # Vetorização TF-IDF das tags do cliente e das receitas
    vectorizer = TfidfVectorizer()
    cliente_vec = vectorizer.fit_transform([cliente_perfil])
    receitas_vec = vectorizer.transform(df_receitas['categories'].apply(lambda x: ' '.join(x)))

    # Cálculo da similaridade de cosseno entre o perfil do cliente e as receitas
    similarity_scores = cosine_similarity(cliente_vec, receitas_vec).flatten()

    # Ordenação das receitas com base na similaridade
    sorted_indices = np.argsort(similarity_scores)[::-1]
    top_receitas_indices = sorted_indices[:top_n]

    # Recomendação das top N receitas para o cliente
    recomendacoes = df_receitas.loc[top_receitas_indices, ['title', 'categories']]
    recomendacoes['score'] = similarity_scores[top_receitas_indices]

    return recomendacoes

Testando recomendações

In [45]:
cliente1_tags = ['vegetarian', 'healthy', 'quick', 'low-carb', 'juice']
cliente1_perfil = criar_perfil(cliente1_tags)
recomendacoes_cliente1 = recomendar_receitas(cliente1_perfil, df, top_n=10)

In [46]:
recomendacoes_cliente1

,title,categories,score
8470,Roasted Carrots with Citrus Vinaigrette,"[Citrus, Side, Bake, Roast, Christmas, Easter,...",0.912871
3068,"Watercress, Endive and Romaine Salad with Feta...","[Salad, Leafy Green, Tomato, Side, No-Cook, Lo...",0.912871
18412,Spiced Peppers and Eggplant,"[Side, Sauté, Low Carb, Vegetarian, Quick & Ea...",0.912871
19806,Sweet Potato Wedges with Lime Mayo,"[Side, Bake, Vegetarian, Kid-Friendly, Quick &...",0.912871
311,Habanero-Orange Salsa,"[Sauce, Side, Vegetarian, Quick & Easy, Low Ca...",0.866025
15718,Habanero-Orange Salsa,"[Sauce, Side, Vegetarian, Quick & Easy, Low Ca...",0.866025
6692,Hummus and Crudités,"[Garlic, Picnic, Vegetarian, Kid-Friendly, Qui...",0.866025
4675,Gluten-Free Vegan Oatmeal Raisin Cookies,"[Cookies, Dessert, Bake, Low Carb, Quick & Eas...",0.866025
8184,Citrus Vinaigrette,"[Citrus, No-Cook, Vegetarian, Quick & Easy, Lo...",0.861640
19149,Fruit Salad and Spritzer,"[Fruit Juice, Non-Alcoholic, Fruit, Nut, Fourt...",0.861640


In [44]:
cliente2_tags = ['gluten-free', 'dessert', 'easy']
cliente2_perfil = criar_perfil(cliente2_tags)
recomendacoes_cliente2 = recomendar_receitas(cliente2_perfil, df)

cliente3_tags = ['low-carb', 'dinner', 'family']
cliente3_perfil = criar_perfil(cliente3_tags)
recomendacoes_cliente3 = recomendar_receitas(cliente3_perfil, df)

In [47]:
recomendacoes_cliente2

,title,categories,score
9520,Almond Macaroons,"[Cookies, Egg, Nut, Dessert, Bake, Quick & Eas...",1.0
9327,Almond Macaroons,"[Cookies, Egg, Nut, Dessert, Bake, Quick & Eas...",1.0
3756,Blueberry Compote,"[Sauce, Fruit, Dessert, Quick & Easy, Low Sodi...",1.0
20123,Blueberry Compote,"[Sauce, Fruit, Dessert, Quick & Easy, Low Sodi...",1.0
8620,"Fresh Fruit Salad with Honey, Mint and Lime Sy...","[Dessert, No-Cook, Quick & Easy, Low Sodium, W...",1.0


In [48]:
recomendacoes_cliente3

,title,categories,score
832,Rosemary-Roasted Chicken and Garlic,"[Chicken, Garlic, Herb, Roast, Low Carb, Quick...",1.000000
3102,White Bean and Pasta Soup,"[Soup/Stew, Bean, Pasta, Sauté, High Fiber, Di...",0.866025
7165,Steak Salad with Herbs,"[Salad, Beef, Herb, Low Carb, Kid-Friendly, Qu...",0.866025
19426,"Roast Chicken with Cumin, Paprika and Allspice","[Chicken, Poultry, Roast, Low Carb, Wheat/Glut...",0.866025
13748,Steamed Clams with Fennel and Spicy Italian Sa...,"[Quick & Easy, Low Cal, Dinner, Sausage, Clam,...",0.866025
